# 🏨 **Hotel Cancel Culture: Who Will Cancel?** 🏨

**Who?**
>* 🏢 **Revenue Management (RM) teams** for hotel groups (corporate, franchise)
>
>
>* 🏨 On-site GMs, Sales, and Ops teams

**Why?**
>* 💰 **Revenue Management:** 
>  * Revenue optimization: Right price, right time, right customer
>    * Dynamic pricing
>    * Distribution channels
>    * Pricing per room type
>
>
>* 🤝 **Sales:**
>  * Group sales (pickup/wash)
>  * BT (performance/company for both GPP and LNR rates)
>
>
>* 🛌 **Rooms Ops:**
>  * Forecasting occupancy, arrivals, departures, stay-overs, same-day booking demand, and probability of guest relocation in the case of oversell.
>  * Determining staff schedules and periods of high demand
>
>
>* 🍰 ☕ **Food and Beverage:**
>  * Ordering food/supplies overall
>  * Scheduling staff
>  * Determining busy times (breakfast, lunch, dinner)
>    * Staffing, specific food/supplies

**What?**
>* 🧾 Dataset comprised of... 
>  * 32 different features
>    * Detailed explanation of features (and sub-categories, when appropriate) available in Readme
>  * Nearly 120,000 reservation records
>  * Source cited in Readme

❌ **How?**
>* Which models/methods? 
>* Data prep and feature engineering

**Goal:** determine whether or not a reservation would cancel given different details of a guest and their reservation.

# 📌 **To-Do**

>* Update each section with ***working*** code
>* Include references/details for relevant packages
---

- [ ] [TD1](#td1)
- [ ] [TD2](#td2)
- [ ] [TD3](#td3)
- [ ] [todo4](#td4)
- [ ] [todo5](#td5)
- [ ] [todo6](#td6)
- [ ] [todo7](#td7)

# 📂 **Imports**

In [13]:
# Data Handling
import pandas as pd
import numpy as np
from scipy import stats

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Modeling - SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

# %load_ext autoreload
# %autoreload 2

import clf_functions.functions as cf

# from sklearn.naive_bayes import MultinomialNB # for naive bayes model

# Settings
%matplotlib inline
plt.style.use('seaborn-talk')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('max_rows', 100)

from sklearn import set_config
set_config(display='diagram')

# 📖 **Read Data**

In [58]:
## Reading data and saving to a DataFrame

data_link = 'http://data.insideairbnb.com/united-states/dc/washington-dc/2021-07-10/data/listings.csv.gz'

data = pd.read_csv(data_link)

In [59]:
data.head(5)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3686,https://www.airbnb.com/rooms/3686,20210710190002,2021-07-11,Vita's Hideaway,IMPORTANT NOTES<br />* Carefully read and be s...,We love that our neighborhood is up and coming...,https://a0.muscache.com/pictures/61e02c7e-3d66...,4645,https://www.airbnb.com/users/show/4645,Vita,2008-11-26,"Washington D.C., District of Columbia, United ...","I am a literary scholar, teacher, poet, vegan ...",within a day,80%,75%,f,https://a0.muscache.com/im/users/4645/profile_...,https://a0.muscache.com/im/users/4645/profile_...,Anacostia,2.00,2.00,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Washington, District of Columbia, United States",Historic Anacostia,nan,38.86,-76.99,Private room in house,Private room,1,nan,1 private bath,1.00,1.00,"[""First aid kit"", ""Long term stays allowed"", ""...",$55.00,2,365,2,2,365,365,2.00,365.00,nan,t,1,31,61,336,2021-07-11,75,3,0,2014-06-22,2021-01-12,4.59,4.71,4.44,4.89,4.82,3.80,4.58,NaN,f,2,0,2,0,0.87
1,3943,https://www.airbnb.com/rooms/3943,20210710190002,2021-07-11,Historic Rowhouse Near Monuments,Please contact us before booking to make sure ...,This rowhouse is centrally located in the hear...,https://a0.muscache.com/pictures/432713/fab7dd...,5059,https://www.airbnb.com/users/show/5059,Vasa,2008-12-12,"Washington, District of Columbia, United States",I have been living and working in DC for the l...,within a few hours,100%,29%,f,https://a0.muscache.com/im/pictures/user/8ec69...,https://a0.muscache.com/im/pictures/user/8ec69...,Eckington,0.00,0.00,"['email', 'phone', 'reviews', 'kba']",t,t,"Washington, District of Columbia, United States","Edgewood, Bloomingdale, Truxton Circle, Eckington",nan,38.91,-77.00,Private room in townhouse,Private room,2,nan,1.5 shared baths,1.00,1.00,"[""Cooking basics"", ""First aid kit"", ""Dedicated...",$70.00,2,1125,2,2,1125,1125,2.00,"1,125.00",nan,t,9,39,69,344,2021-07-11,429,0,0,2010-08-08,2018-08-07,4.82,4.89,4.91,4.94,4.90,4.54,4.74,NaN,f,2,0,2,0,3.22
2,4529,https://www.airbnb.com/rooms/4529,20210710190002,2021-07-11,Bertina's House Part One,This is large private bedroom with plenty of...,Very quiet neighborhood and it is easy accessi...,https://a0.muscache.com/pictures/86072003/6709...,5803,https://www.airbnb.com/users/show/5803,Bertina'S House,2008-12-30,"Washington, District of Columbia, United States","I am an easy going, laid back person who loves...",NaN,NaN,NaN,f,https://a0.muscache.com/im/users/5803/profile_...,https://a0.muscache.com/im/users/5803/profile_...,Eastland Gardens,3.00,3.00,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"Washington, District of Columbia, United States","Eastland Gardens, Kenilworth",nan,38.91,-76.94,Private room

In [60]:
data.shape

(8033, 74)

> The initial reading of the dataset shows there are 74 features and 8,033 entries. A quick glance at the `.head()` gives a sample of the entries, showing that some of the features are not relevant to my analysis.
>
> I will drop the following features now as they are metadata related to the webscraping required to provide the data (and would not have any meaningful impact on my analysis):
>
>* listing_url
>
>
>* scrape_id
>
>
>* last_scraped
>
>
>* picture_url
>
>
>* host_url
>
>
>* host_thumbnail_url
>
>
>* host_picture_url
>
>
>* calendar_last_scraped

In [62]:
df = data.drop(columns= ['listing_url', 'scrape_id', 'last_scraped', 'picture_url',
                         'host_url', 'host_thumbnail_url','host_picture_url',
                         'calendar_last_scraped']).copy()

In [63]:
## Creating features/target for dataset
target = 'review_scores_rating'

X = df.drop(columns = target).copy()
y = df[target].copy()

In [64]:
## Confirming same number of rows
X.shape[0] == y.shape[0]

True

In [65]:
## Splitting to prevent data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 🧼 **Starting Cleaning and EDA**

In [70]:
## Checking for class imbalances
y_train.describe()

count   4,406.00
mean        4.69
std         0.69
min         0.00
25%         4.68
50%         4.86
75%         5.00
max         5.00
Name: review_scores_rating, dtype: float64

> The target feature, `'review_scores_rating'`, is currently a range of values from 0 to 5, with 75% of the scores being 4.68 or above. In order to use these values, I would need to bin them into categories based on ranges of values for analysis.

From original article:

> ❌

In [17]:
report = cf.report_df(X_train)
report

(6024, 73)


,datatypes,num_unique,null_sum,null_pct,count,mean,std,min,25%,50%,75%,max
id,int64,6024,0,0.00,"6,024.00","29,250,080.54","14,822,121.69","3,943.00","16,612,253.75","28,574,240.50","43,680,464.50","50,949,841.00"
listing_url,object,6024,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan
scrape_id,int64,1,0,0.00,"6,024.00","20,210,710,190,000.30",1.70,"20,210,710,190,002.00","20,210,710,190,002.00","20,210,710,190,002.00","20,210,710,190,002.00","20,210,710,190,002.00"
last_scraped,object,3,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan
name,object,5700,1,0.00,nan,nan,nan,nan,nan,nan,nan,nan
description,object,5401,122,0.02,nan,nan,nan,nan,nan,nan,nan,nan
neighborhood_overview,object,3268,2173,0.36,nan,nan,nan,nan,nan,nan,nan,nan
picture_url,object,5779,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan
host_id,int64,3762,0,0.00,"6,024.00","96,645,835.00","111,765,783.21","4,492.00","14,136,155.50","48,005,494.00","135,378,552.00","411,330,714.00"
host_url,object,3762,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan


In [23]:
# df2.style.bar(subset=['A', 'B'], color='#d65f5f')

report.style.bar(subset=['null_pct'], color='#d65f5f')#, vmin=0, vmax=1)

,datatypes,num_unique,null_sum,null_pct,count,mean,std,min,25%,50%,75%,max
id,int64,6024,0,0.000000,6024.000000,29250080.542995,14822121.694414,3943.000000,16612253.750000,28574240.500000,43680464.500000,50949841.000000
listing_url,object,6024,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan
scrape_id,int64,1,0,0.000000,6024.000000,20210710190000.296875,1.703266,20210710190002.000000,20210710190002.000000,20210710190002.000000,20210710190002.000000,20210710190002.000000
last_scraped,object,3,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan
name,object,5700,1,0.000166,nan,nan,nan,nan,nan,nan,nan,nan
description,object,5401,122,0.020252,nan,nan,nan,nan,nan,nan,nan,nan
neighborhood_overview,object,3268,2173,0.360724,nan,nan,nan,nan,nan,nan,nan,nan
picture_url,object,5779,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan
host_id,int64,3762,0,0.000000,6024.000000,96645834.999834,111765783.205877,4492.000000,14136155.500000,48005494.000000,135378552.000000,411330714.000000
host_url,object,3762,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan


In [33]:
from ipywidgets import interact

In [55]:
@interact
def sort_report(Column=['num_unique', 'null_pct'], ascending = False):
    display(report.sort_values(Column, ascending=ascending).style.bar(subset=[Column], color='#d65f5f'))

interactive(children=(Dropdown(description='Column', options=('num_unique', 'null_pct'), value='num_unique'), …

In [54]:
import fsds as fs

In [47]:
fs.ihelp_menu([fs.ihelp_menu, sort_report])

GridBox(box_style='warning', children=(VBox(children=(Label(value='iHelp Menu: View Help and/or Source Code'),…

Output()

In [32]:
sort_report('null_sum', ascending=True)

,datatypes,num_unique,null_sum,null_pct,count,mean,std,min,25%,50%,75%,max
id,int64,6024,0,0.000000,6024.000000,29250080.542995,14822121.694414,3943.000000,16612253.750000,28574240.500000,43680464.500000,50949841.000000
number_of_reviews,int64,346,0,0.000000,6024.000000,36.675631,68.936515,0.000000,0.000000,6.000000,40.000000,662.000000
latitude,float64,4230,0,0.000000,6024.000000,38.910535,0.024833,38.820280,38.897435,38.910440,38.923720,38.996570
longitude,float64,4478,0,0.000000,6024.000000,-77.017509,0.029381,-77.112840,-77.036825,-77.019845,-76.998038,-76.910950
property_type,object,46,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan
room_type,object,4,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan
accommodates,int64,17,0,0.000000,6024.000000,3.541169,2.222244,0.000000,2.000000,3.000000,4.000000,16.000000
calendar_last_scraped,object,3,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan
calculated_host_listings_count_shared_rooms,int64,12,0,0.000000,6024.000000,0.282537,2.028975,0.000000,0.000000,0.000000,0.000000,20.000000
amenities,object,5319,0,0.000000,nan,nan,nan,nan,nan,nan,nan,nan


## ❓ **Null Values and How to Address:**

>* **Children:** fill in missing values with zero (75% of the data is zero, making it an easy and accurate choice)
>
>
>* **Country, Agent:** For these three sections, the documentation for the dataset indicates that "NULL" or NaN values are not truly *missing* from the dataset, but that the data source did not have any values for those reservations.
>  * **To address:** I will impute the string, "missing," as a placeholder to keep these features in the dataset.
>
>
>* **Company:** Since this feature is 94% empty values, it would have an insignificant impact on the predictions of whether or not a guest will cancel their reservation.
>  * **To address:** I will drop the column as part of the 

In [9]:
X_train.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
118418,City Hotel,208,2017,August,33,17,0,3,2,0.00,0,BB,DEU,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,131.70,1,0,Check-Out,2017-08-20
66950,City Hotel,110,2017,April,17,26,1,4,2,0.00,0,SC,NOR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,100.80,0,0,Canceled,2017-01-06
52240,City Hotel,330,2016,June,23,2,0,2,2,0.00,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,1.00,nan,0,Transient,62.00,0,0,Canceled,2015-10-21
3835,Resort Hotel,3,2016,January,4,21,0,1,1,0.00,0,BB,PRT,Corporate,Corporate,0,0,0,A,E,0,No Deposit,nan,47.00,0,Transient,30.00,0,0,Canceled,2016-01-18
3330,Resort Hotel,66,2015,December,49,5,1,1,2,0.00,0,FB,PRT,Groups,TA/TO,0,0,0,A,A,0,No Deposit,155.00,nan,0,Transient-Party,64.00,0,0,Canceled,2015-12-02


In [10]:
X_train['meal'].value_counts()

BB           69242
HB           10794
SC            8027
Undefined      888
FB             591
Name: meal, dtype: int64

In [11]:
und_count = X_train.isin(['Undefined']).sum()
# und_count

In [12]:
und_count[und_count > 0]

meal                    888
market_segment            2
distribution_channel      4
dtype: int64

#### Old code - saving just in case

In [13]:
## Unsure what this code was for - something about the "Undefined" values? 

# for col in X_train.columns:
#     if X_train[col].dtype == 'O':
#         print(X_train[col].value_counts(),'\n\n')

In [14]:
# for col in X_train.columns:
#     if 'Undefined' in X_train[col]:
#         print(col)

In [15]:
# 'undefined' in X_train["meal"]

In [16]:
# X_train['distribution_channel'].unique()

In [17]:
# #3 Identifying which columns indicate missing values based on .isna() method

# nan_cols = []

# for column in X_train.columns:
#     if X_train[column].isna().sum() > 0:
#         nan_cols.append(column)
#         print(column,'\n\n',X_train[column].value_counts(dropna=False),'\n\n')
        
# nan_cols

In [18]:
# for column in X_train.columns:
#     if X_train[column].isna().sum() > 0:
# #         nan_cols.append(column)
#         print(column,'\n\n',X_train[column].isna().sum(),'\n\n')

In [19]:
# X_train['country'].isna().sum()

In [20]:
# X_train[X_train['country'].isna()]

# 🚿 **Preprocessing Pipeline**

In [23]:
cat_cols = ['hotel', 'meal','arrival_date_month', 'country', 'market_segment',
            'distribution_channel','is_repeated_guest','reserved_room_type',
            'assigned_room_type','deposit_type', 'agent',
            'customer_type','reservation_status']

cont_cols = [col for col in X_train.drop(['reservation_status_date','company'],axis=1).columns if col not in cat_cols]

cont_cols

['lead_time',
 'arrival_date_year',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests']

In [24]:
X_train[cat_cols] = X_train[cat_cols].astype(str)

C:\Users\bmcca\anaconda3\envs\learn-env-bmc\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [25]:
X_test[cat_cols] = X_test[cat_cols].astype(str)

In [26]:
## Creating ColumnTransformer and sub-transformers for imputation and encoding

# Filling missing "Children"
zero_transformer = SimpleImputer(strategy='constant', fill_value=0)

##  
missing_transformer = SimpleImputer(strategy='constant', fill_value='missing')

## Encoding categoricals - handling errors to prevent issues w/ test set
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

cat_pipe = Pipeline(steps=[('imputer', missing_transformer),
                      ('ohe', categorical_transformer)])

cont_pipe = Pipeline(steps=[('imputer', zero_transformer),
                           ('scaler', StandardScaler())])

## Instantiating the ColumnTransformer and including all transformers
preprocessor = ColumnTransformer(
    transformers=[('conts', cont_pipe, cont_cols),
                  ('cats', cat_pipe, cat_cols)])

preprocessor

ColumnTransformer(transformers=[('conts',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant')),
                                                 ('scaler', StandardScaler())]),
                                 ['lead_time', 'arrival_date_year',
                                  'arrival_date_week_number',
                                  'arrival_date_day_of_month',
                                  'stays_in_weekend_nights',
                                  'stays_in_week_nights', 'adults', 'children',
                                  'babies', 'previous_cancellations',
                                  'previous_bo...
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['hotel', 'meal', 'arrival_date_month',
                                  'country', 'market_segment',
                                  'distribution_channel', 'is_repeated_guest',
                                  'reserved_room_type', 'assigned_room_type',
                                  'deposit_type', 'agent', 'customer_type',
                                  'reservation_status'])])

In [27]:
preprocessor.fit(X_train)

## Getting feature names from OHE
ohe_cat_names = preprocessor.named_transformers_['cats'].named_steps['ohe'].get_feature_names(cat_cols)

In [28]:
## Generating list for column index
final_cols = [*cont_cols, *ohe_cat_names]

## Fit and transform the data via the ColumnTransformer
X_train_tf = preprocessor.transform(X_train)
X_train_tf_df = pd.DataFrame(X_train_tf, columns=final_cols, index=X_train.index)

## Transforming the test set and saving
X_test_tf = preprocessor.transform(X_test)
X_test_tf_df = pd.DataFrame(X_test_tf, columns=final_cols, index=X_test.index)

display(X_train_tf_df.head(5),X_test_tf_df.head(5))

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,country_ABW,country_AGO,country_AIA,country_ALB,country_AND,country_ARE,country_ARG,country_ARM,country_ASM,country_ATA,country_ATF,country_AUS,country_AUT,country_AZE,country_BDI,country_BEL,country_BEN,country_BFA,country_BGD,country_BGR,country_BHR,country_BHS,country_BIH,country_BLR,country_BOL,country_BRA,country_BRB,country_BWA,country_CAF,country_CHE,country_CHL,country_CHN,country_CIV,country_CMR,country_CN,country_COL,country_COM,country_CPV,country_CRI,country_CUB,country_CYM,country_CYP,country_CZE,country_DEU,country_DMA,country_DNK,country_DOM,country_DZA,country_ECU,country_EGY,country_ESP,country_EST,country_ETH,country_FIN,country_FJI,country_FRA,country_FRO,country_GAB,country_GBR,country_GEO,country_GGY,country_GHA,country_GIB,country_GLP,country_GNB,country_GRC,country_GTM,country_GUY,country_HKG,country_HRV,country_HUN,country_IDN,country_IMN,country_IND,country_IRL,country_IRN,country_IRQ,country_ISL,country_ISR,country_ITA,country_JAM,country_JEY,country_JOR,country_JPN,country_KAZ,country_KEN,country_KHM,country_KIR,country_KNA,country_KOR,country_KWT,country_LAO,country_LBN,country_LBY,country_LIE,country_LKA,country_LTU,country_LUX,country_LVA,country_MAC,country_MAR,country_MCO,country_MDG,country_MDV,country_MEX,country_MKD,country_MLI,country_MLT,country_MMR,country_MNE,country_MOZ,country_MUS,country_MWI,country_MYS,country_NAM,country_NCL,country_NGA,country_NIC,country_NLD,country_NOR,country_NPL,country_NZL,country_OMN,country_PAK,country_PAN,country_PER,country_PHL,country_PLW,country_POL,country_PRI,country_PRT,country_PRY,country_PYF,country_QAT,country_ROU,country_RUS,country_RWA,country_SAU,country_SDN,country_SEN,country_SGP,country_SLE,country_SLV,country_SMR,country_SRB,country_STP,country_SUR,country_SVK,country_SVN,country_SWE,country_SYC,country_SYR,country_TGO,country_THA,country_TJK,country_TMP,country_TUN,country_TUR,country_TWN,country_TZA,country_UGA,country_UKR,country_UMI,country_URY,country_USA,country_UZB,country_VEN,country_VGB,country_VNM,country_ZAF,country_ZMB,country_ZWE,country_nan,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,is_repeated_guest_0,is_repeated_guest_1,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,agent_1.0,agent_10.0,agent_103.0,agent_104.0,agent_105.0,agent_106.0,agent_107.0,agent_11.0,agent_110.0,agent_111.0,agent_112.0,agent_114.0,agent_115.0,agent_118.0,agent_119.0,agent_12.0,agent_121.0,agent_126.0,agent_127.0,agent_128.0,agent_129.0,agent_13.0,agent_132.0,agent_133.0,agent_134.0,agent_135.0,agent_138.0,agent_139.0,agent

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,country_ABW,country_AGO,country_AIA,country_ALB,country_AND,country_ARE,country_ARG,country_ARM,country_ASM,country_ATA,country_ATF,country_AUS,country_AUT,country_AZE,country_BDI,country_BEL,country_BEN,country_BFA,country_BGD,country_BGR,country_BHR,country_BHS,country_BIH,country_BLR,country_BOL,country_BRA,country_BRB,country_BWA,country_CAF,country_CHE,country_CHL,country_CHN,country_CIV,country_CMR,country_CN,country_COL,country_COM,country_CPV,country_CRI,country_CUB,country_CYM,country_CYP,country_CZE,country_DEU,country_DMA,country_DNK,country_DOM,country_DZA,country_ECU,country_EGY,country_ESP,country_EST,country_ETH,country_FIN,country_FJI,country_FRA,country_FRO,country_GAB,country_GBR,country_GEO,country_GGY,country_GHA,country_GIB,country_GLP,country_GNB,country_GRC,country_GTM,country_GUY,country_HKG,country_HRV,country_HUN,country_IDN,country_IMN,country_IND,country_IRL,country_IRN,country_IRQ,country_ISL,country_ISR,country_ITA,country_JAM,country_JEY,country_JOR,country_JPN,country_KAZ,country_KEN,country_KHM,country_KIR,country_KNA,country_KOR,country_KWT,country_LAO,country_LBN,country_LBY,country_LIE,country_LKA,country_LTU,country_LUX,country_LVA,country_MAC,country_MAR,country_MCO,country_MDG,country_MDV,country_MEX,country_MKD,country_MLI,country_MLT,country_MMR,country_MNE,country_MOZ,country_MUS,country_MWI,country_MYS,country_NAM,country_NCL,country_NGA,country_NIC,country_NLD,country_NOR,country_NPL,country_NZL,country_OMN,country_PAK,country_PAN,country_PER,country_PHL,country_PLW,country_POL,country_PRI,country_PRT,country_PRY,country_PYF,country_QAT,country_ROU,country_RUS,country_RWA,country_SAU,country_SDN,country_SEN,country_SGP,country_SLE,country_SLV,country_SMR,country_SRB,country_STP,country_SUR,country_SVK,country_SVN,country_SWE,country_SYC,country_SYR,country_TGO,country_THA,country_TJK,country_TMP,country_TUN,country_TUR,country_TWN,country_TZA,country_UGA,country_UKR,country_UMI,country_URY,country_USA,country_UZB,country_VEN,country_VGB,country_VNM,country_ZAF,country_ZMB,country_ZWE,country_nan,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,is_repeated_guest_0,is_repeated_guest_1,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,agent_1.0,agent_10.0,agent_103.0,agent_104.0,agent_105.0,agent_106.0,agent_107.0,agent_11.0,agent_110.0,agent_111.0,agent_112.0,agent_114.0,agent_115.0,agent_118.0,agent_119.0,agent_12.0,agent_121.0,agent_126.0,agent_127.0,agent_128.0,agent_129.0,agent_13.0,agent_132.0,agent_133.0,agent_134.0,agent_135.0,agent_138.0,agent_139.0,agent

# 📝 Next Steps

* Process classification model - i.e. Logreg, KNN, DecisionTrees, etc.
* Evaluate results
* Determine if I need to redo pre-processing steps

# 🚿 Classification Pipeline